<a href="https://colab.research.google.com/github/EMbeDS-education/ComputingDataAnalysisModeling20242025/blob/main/ISE/SearchEngineWeaviate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Weaviate as a Search Engine

##################### TO START

Please download your COHERE key to vectorize texts from

https://dashboard.cohere.com/api-keys.

Then load your key in the "secrets" of Colab, here to the left, by clicking on the key icon, and storing as "COHERE_APIKEY".

You find all the documentation of Weaviate in:

https://weaviate.io/developers/weaviate

###########################################################

In [1]:
!pip install -U weaviate-client
import weaviate
import weaviate.classes.config as wc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.9/353.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.8/223.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.0 MB/s eta 0:00:00


In [2]:
# Create the Weaviate client

import weaviate
from weaviate.classes.query import MetadataQuery
from weaviate.classes.config import Configure, Property, DataType, Tokenization
from weaviate.classes.query import Filter

client = weaviate.connect_to_embedded()

INFO:weaviate-client:Binary /root/.cache/weaviate-embedded did not exist. Downloading binary from https://github.com/weaviate/weaviate/releases/download/v1.26.6/weaviate-v1.26.6-Linux-amd64.tar.gz
INFO:weaviate-client:Started /root/.cache/weaviate-embedded: process ID 1037


In [3]:
!wget https://raw.githubusercontent.com/EMbeDS-education/ComputingDataAnalysisModeling20242025/refs/heads/main/ISE/data/5articles.json
import json

with open("5articles.json", 'r') as f:
  articles = json.load(f)

articles[0]

--2025-03-30 13:44:51--  https://raw.githubusercontent.com/EMbeDS-education/ComputingDataAnalysisModeling20242025/refs/heads/main/ISE/data/5articles.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12566 (12K) [text/plain]
Saving to: ‘5articles.json’

5articles.json      100%[===================>]  12.27K  --.-KB/s    in 0s      

2025-03-30 13:44:51 (58.7 MB/s) - ‘5articles.json’ saved [12566/12566]



{'title': 'American Airlines orders 60 Overture supersonic jets',
 'maintext': "The revival of supersonic passenger travel, thought to be long dead with the demise of Concorde nearly two decades ago, could be about to take wing as American Airlines has put in an order for 60 aircraft capable of flying at 1.7 times the speed of sound. \nBoom is a start-up based in Denver, Colorado, whose development of Overture, an ultra-fast successor to Concorde that seats 65 to 88 passengers, is so advanced that it showed off designs at last month's Farnborough air show.",
 'date': '2022-08-18',
 'source': 'The New York Times'}

Let's create a simple collection that has two fields of type TEXT and call it "TestCollection".

**TOKENIZATION OPTIONS**
* word: alphanumeric, lowercased tokens, with stopwords filtering (default tokenizer for Weaviate)
* lowercase: lowercased tokens
* whitespace: whitespace-separated, case-sensitive tokens
* field: the entire value of the property is treated as a single token

[All property types](https://weaviate.io/developers/weaviate/config-refs/datatypes)

In [4]:
client.collections.delete_all() # To remove anything created before, if any

client.collections.create(
    name="TestCollection",
    properties=[
        wc.Property(name="maintext", data_type=wc.DataType.TEXT, tokenization=Tokenization.WORD),
        wc.Property(name="title", data_type=wc.DataType.TEXT, tokenization=Tokenization.LOWERCASE),
        wc.Property(name="source", data_type=wc.DataType.TEXT, tokenization=Tokenization.FIELD),
        wc.Property(name="date", data_type=wc.DataType.DATE)
    ],
)

# Just typing "wc.DataType." you can explore other data types...

**CREATE A TEST COLLECTION**

Now we create the collection of name "TestCollection" and we insert the "articles" in it

In [5]:
from datetime import timezone, datetime

documents = client.collections.get("TestCollection")

for doc in articles:
    documents.data.insert({
        "maintext": doc["maintext"],
        "title": doc["title"],
        "source": doc["source"],
        "date": datetime.strptime(doc["date"], "%Y-%m-%d").replace(tzinfo=timezone.utc)
    })


**ITERATE over all documents in the collection**

Notice that it is a "map" so that the listing is not necessarily in the order in which they have been inserted

In [6]:
# retrieve the elements
for i, doc in enumerate(documents.iterator()):
  print(doc.uuid, " - ", doc.properties["title"], " - ", doc.properties["source"], " - ", doc.properties["date"])

0fae5c48-c330-4114-a82e-043d9ab69163  -  American Airlines orders 60 Overture supersonic jets  -  The New York Times  -  2022-08-18 00:00:00+00:00
6f7fd93f-fa2f-4fc2-a4ca-9f4186ad47ad  -  Leclerc dedicates win to Hubert  -  The Herald-ir  -  2019-09-01 00:00:00+00:00
928ba648-2c58-4862-835a-533ea64ced00  -  'One-punch killer's sentence will make others think twice'  -  The Herald-ir  -  2019-06-29 00:00:00+00:00
a18a6b88-1ca8-49a9-8734-b5b8475d3508  -  Gunman opens fire on car just metres from scene of Hamid Sanambar murder  -  The Herald-ir  -  2019-06-07 00:00:00+00:00
fb9e8601-1ff4-48ca-ba14-ba202cf83abb  -  Conte: 'Chelsea are not in the race to sign Sanchez'  -  The Herald-ir  -  2018-01-23 00:00:00+00:00


**QUERYING THE COLLECTION**

Let's try some simple queries, **bm25** is the sparse vectorization of texts (better than TFIDF) that Weaviate adopts.

Notice that it lowercases the parsed tokens, but it does not stem them. This is on the roadmap of features that Weaviate plans to support in the future.

In [7]:
query = "race"
response = documents.query.bm25(query=query, return_metadata=MetadataQuery(score=True))
for o in response.objects:
    print("{} - {}".format(round(o.metadata.score*100)/100, o.properties["title"]))
    print("query term occurs: ", o.properties["maintext"].count("race"), " in maintext and ", o.properties["title"].count("race"), " in title\n")

1.18 - Conte: 'Chelsea are not in the race to sign Sanchez'
query term occurs:  1  in maintext and  1  in title

0.46 - Leclerc dedicates win to Hubert
query term occurs:  3  in maintext and  0  in title



In [8]:
# parsing is induced also by -

query = "punch"
response = documents.query.bm25(query=query, return_metadata=MetadataQuery(score=True))
for o in response.objects:
    print("{} - {}".format(round(o.metadata.score*100)/100, o.properties["title"]))

0.7 - 'One-punch killer's sentence will make others think twice'


In [9]:
# parsing is lowercase

query = "sanchez"
response = documents.query.bm25(query=query, return_metadata=MetadataQuery(score=True))
for o in response.objects:
    print("{} - {}".format(round(o.metadata.score*100)/100, o.properties["title"]))

0.77 - Conte: 'Chelsea are not in the race to sign Sanchez'


In [10]:
# It is a FIELD parsing, so it does not find "The New York Times" in article on "American Airlines"

query = "The"
response = documents.query.bm25(query=query, return_metadata=MetadataQuery(score=True))
for o in response.objects:
    print("{} - {}".format(round(o.metadata.score*100)/100, o.properties["title"]))

0.97 - Conte: 'Chelsea are not in the race to sign Sanchez'


In [11]:
# the stopwords are not present by assuming English in field "maintext" but they are include in the field "title"

query = "the"
response = documents.query.bm25(query=query, return_metadata=MetadataQuery(score=True))
for o in response.objects:
    print("{} - {}".format(round(o.metadata.score*100)/100, o.properties["title"]))

0.97 - Conte: 'Chelsea are not in the race to sign Sanchez'


**CHANGING STOPWORDS**

Say that you now want to consider some words as "stopwords", that the system does not consider as such by default

documents.config.update(inverted_index_config=wc.Reconfigure.inverted_index(stopwords_additions=["victory"]))



**SEARCH ON A SPECIFIC PROPERTY**

In [12]:
# maintext == is tokenized as WORD and thus stopwords are removed (hence, "will" is removed)

response = documents.query.bm25(
    query="will",
    query_properties=["maintext"], # this is the line to add
    return_metadata=MetadataQuery(score=True)
)
for o in response.objects:
    print("{} - {}".format(round(o.metadata.score*100)/100, o.properties["title"]))


In [13]:
# title == is tokenized as LOWERCASE and thus stopwords are NOT removed


response = documents.query.bm25(
    query="will",
    query_properties=["title"], # this is the line to add
    return_metadata=MetadataQuery(score=True)
)
for o in response.objects:
    print("{} - {}".format(round(o.metadata.score*100)/100, o.properties["title"]))
    print("query term occurs: ", o.properties["title"].count("will"), "\n")

0.65 - 'One-punch killer's sentence will make others think twice'
query term occurs:  1 



**AFTER FIELD BOOSTING**

The score is not twice. It applies some normalization and other formulas.

In [14]:
print("BEFORE BOOSTING:\n")
response = documents.query.bm25(
    query="race",
    query_properties=["title", "maintext"],
    return_metadata=MetadataQuery(score=True)
)
for o in response.objects:
    print("{} - {}".format(round(o.metadata.score*100)/100, o.properties["title"]))

print("\n\nAFTER BOOSTING:\n")
response = documents.query.bm25(
    query="race",
    query_properties=["title^2", "maintext"],
    return_metadata=MetadataQuery(score=True)
)
for o in response.objects:
    print("{} - {}".format(round(o.metadata.score*100)/100, o.properties["title"]))

BEFORE BOOSTING:

1.27 - Conte: 'Chelsea are not in the race to sign Sanchez'
0.54 - Leclerc dedicates win to Hubert


AFTER BOOSTING:

1.43 - Conte: 'Chelsea are not in the race to sign Sanchez'
0.54 - Leclerc dedicates win to Hubert


**APPLY FIELD FILTERING**

In [15]:
response = documents.query.bm25(
    query="race",
    query_properties=["title^2", "maintext"],
    filters=Filter.by_property("title").contains_any(["Leclerc", "formula"]),
    return_metadata=MetadataQuery(score=True)
)
for o in response.objects:
    print("{} - {}".format(round(o.metadata.score*100)/100, o.properties["title"]))

0.54 - Leclerc dedicates win to Hubert


**FILTERING BY DATE or MANY PROPERTIES**

In [16]:
print("BEFORE FILTERING BY DATE:\n")
response = documents.query.bm25(
    query="race",
    return_metadata=MetadataQuery(score=True)
)
for o in response.objects:
    print("{} - {} ({})".format(round(o.metadata.score*100)/100, o.properties["title"], o.properties["date"]))

print("\n\nAFTER FILTERING BY DATE:\n")
reference_date = datetime.strptime("2018-08-15", "%Y-%m-%d").replace(tzinfo=timezone.utc)
response = documents.query.bm25(
    query="race",
    filters=Filter.by_property("date").greater_or_equal(reference_date),
    return_metadata=MetadataQuery(score=True)
)
for o in response.objects:
    print("{} - {} ({})".format(round(o.metadata.score*100)/100, o.properties["title"], o.properties["date"]))


print("\n\nAPPLY MANY FILTERS OVER MANY PROPERTIES:\n")
reference_date = datetime.strptime("2018-08-15", "%Y-%m-%d").replace(tzinfo=timezone.utc)
response = documents.query.bm25(
    query="race",
    filters=( ## use of &, but also |
        Filter.by_property("date").greater_or_equal(reference_date) &
        Filter.by_property("title").contains_any(["won", "formula"])
        ),
    return_metadata=MetadataQuery(score=True)
)
for o in response.objects:
    print("{} - {} ({})".format(round(o.metadata.score*100)/100, o.properties["title"], o.properties["date"]))


BEFORE FILTERING BY DATE:

1.18 - Conte: 'Chelsea are not in the race to sign Sanchez' (2018-01-23 00:00:00+00:00)
0.46 - Leclerc dedicates win to Hubert (2019-09-01 00:00:00+00:00)


AFTER FILTERING BY DATE:

0.46 - Leclerc dedicates win to Hubert (2019-09-01 00:00:00+00:00)


APPLY MANY FILTERS OVER MANY PROPERTIES:



**VECTORIZATION:: DENSE EMBEDDINGS**

Some advanced features, let's try some vectorized queries. Let's assume we want to find all articles that are "related to sport". In this current collection, "sport" is not present as a word in any title or maintext.

In [17]:
# The term "sport" does not occur and so BM25 does not return any result

response = documents.query.bm25(query="sport", return_metadata=MetadataQuery(score=True))
for o in response.objects:
    print("{} - {}".format(round(o.metadata.score*100)/100, o.properties["title"]))

In [18]:
# Unfortunately, we cannot use all the vectorizer modules that are present in Weaviate.
# Here is a list of the ones that are available
client.get_meta()

{'hostname': 'http://127.0.0.1:8079',
 'modules': {'generative-openai': {'documentationHref': 'https://platform.openai.com/docs/api-reference/completions',
   'name': 'Generative Search - OpenAI'},
  'qna-openai': {'documentationHref': 'https://platform.openai.com/docs/api-reference/completions',
   'name': 'OpenAI Question & Answering Module'},
  'ref2vec-centroid': {},
  'reranker-cohere': {'documentationHref': 'https://txt.cohere.com/rerank/',
   'name': 'Reranker - Cohere'},
  'text2vec-cohere': {'documentationHref': 'https://docs.cohere.ai/embedding-wiki/',
   'name': 'Cohere Module'},
  'text2vec-huggingface': {'documentationHref': 'https://huggingface.co/docs/api-inference/detailed_parameters#feature-extraction-task',
   'name': 'Hugging Face Module'},
  'text2vec-openai': {'documentationHref': 'https://platform.openai.com/docs/guides/embeddings/what-are-embeddings',
   'name': 'OpenAI Module'}},
 'version': '1.26.6'}

Let's use COHERE as a textual vectorizer [https://dashboard.cohere.com/api-keys](https://dashboard.cohere.com/api-keys). As we can see, using colab we have only a few options for vectorization (openai, cohere, huggingface). Additionally, only one generation model is available (openai).
Cohere provides free sample apis. OpenAI does not.

At this link you find how to integrate MODELS [https://weaviate.io/developers/weaviate/model-providers](https://weaviate.io/developers/weaviate/model-providers)

In [19]:
## You need first to create a KEY !!!!
from google.colab import userdata

client.close()
cohere_key = userdata.get('COHERE_APIKEY') # MAKE SURE YOU CREATED A KEY
headers = {
    "X-Cohere-Api-Key": cohere_key,
}
client = weaviate.connect_to_embedded(headers=headers)

INFO:weaviate-client:Started /root/.cache/weaviate-embedded: process ID 1206


**CREATE A VECTOR DB**


In [20]:
client.collections.delete_all()
client.collections.create(
    name="TestVectorizer",
    properties=[
        wc.Property(name="maintext", data_type=wc.DataType.TEXT, tokenization=Tokenization.WORD),
        wc.Property(name="title", data_type=wc.DataType.TEXT, tokenization=Tokenization.LOWERCASE),
        wc.Property(name="source", data_type=wc.DataType.TEXT, tokenization=Tokenization.FIELD),
        wc.Property(name="date", data_type=wc.DataType.DATE)
    ],
    vectorizer_config=[
        Configure.NamedVectors.text2vec_cohere(
            name="maintext_vector",
            source_properties=["maintext"],
            #model="embed-multilingual-light-v3.0"
        )
    ]
)

In [21]:
from datetime import timezone, datetime

documents = client.collections.get("TestVectorizer")

for doc in articles:
    documents.data.insert({
        "maintext": doc["maintext"],
        "title": doc["title"],
        "source": doc["source"],
        "date": datetime.strptime(doc["date"], "%Y-%m-%d").replace(tzinfo=timezone.utc)
    })


In [22]:
print("pure syntactical search (ordered by decreasing similarity score): 'sport'\n")
response = documents.query.bm25(query="sport", return_metadata=MetadataQuery(score=True))
for o in response.objects:
    print("{} - {}".format(round(o.metadata.score*100)/100, o.properties["title"]))

pure syntactical search (ordered by decreasing similarity score): 'sport'



In [23]:
print("pure vector search (ordered by increasing distance): 'sport'\n")
# NOTE THAT WE ALSO NEED THE PARAMETER DISTANCE
response = documents.query.near_text(query="sport", return_metadata=MetadataQuery(score=True, distance=True), limit=3)
for o in response.objects:
  print("{} - {} (score is {})".format(round(o.metadata.distance*100)/100, o.properties["title"], round(o.metadata.score*100)/100))

pure vector search (ordered by increasing distance): 'sport'

0.6 - Leclerc dedicates win to Hubert (score is 0.0)
0.61 - Gunman opens fire on car just metres from scene of Hamid Sanambar murder (score is 0.0)
0.65 - Conte: 'Chelsea are not in the race to sign Sanchez' (score is 0.0)


**HYBRID SEARCH**

Let us consider a query that admits results for both syntactic and vectorized search

In [24]:
print("pure syntactical search (ordered by decreasing similarity score): 'race'\n")
response = documents.query.bm25(query="race", return_metadata=MetadataQuery(score=True))
for o in response.objects:
    print("{} - {}".format(round(o.metadata.score*100)/100, o.properties["title"]))

pure syntactical search (ordered by decreasing similarity score): 'race'

1.18 - Conte: 'Chelsea are not in the race to sign Sanchez'
0.46 - Leclerc dedicates win to Hubert


In [25]:
print("pure vector search (ordered by increasing distance): 'race'\n")
# NOTE THAT WE ALSO NEED THE PARAMETER DISTANCE
response = documents.query.near_text(query="race", return_metadata=MetadataQuery(score=True, distance=True), limit=3)
for o in response.objects:
  print("{} - {} (score is {})".format(round(o.metadata.distance*100)/100, o.properties["title"], round(o.metadata.score*100)/100))

pure vector search (ordered by increasing distance): 'race'

0.6 - Leclerc dedicates win to Hubert (score is 0.0)
0.61 - Gunman opens fire on car just metres from scene of Hamid Sanambar murder (score is 0.0)
0.69 - Conte: 'Chelsea are not in the race to sign Sanchez' (score is 0.0)


In [26]:
# 0 = syntactic, 1 = vectorized
print("hybrid search (ordered by decreasing score): 'race'")
response = documents.query.hybrid(query="race", alpha=0.5, return_metadata=MetadataQuery(score=True, explain_score=True), limit=3)
for o in response.objects:
  print("{} - {} [{}]".format(round(o.metadata.score*100)/100, o.properties["title"],  o.metadata.explain_score.strip().replace("\n", '')))

hybrid search (ordered by decreasing score): 'race'
0.6 - Conte: 'Chelsea are not in the race to sign Sanchez' [Hybrid (Result Set keyword,bm25) Document f436452f-bbe9-4409-b97b-910d6f18ea6c: original score 1.1806082, normalized score: 0.5 - Hybrid (Result Set vector,hybridVector) Document f436452f-bbe9-4409-b97b-910d6f18ea6c: original score 0.31257296, normalized score: 0.10023565]
0.5 - Leclerc dedicates win to Hubert [Hybrid (Result Set keyword,bm25) Document dbab6769-b92c-43a7-b28e-9e266e575b1d: original score 0.46220398, normalized score: 0 - Hybrid (Result Set vector,hybridVector) Document dbab6769-b92c-43a7-b28e-9e266e575b1d: original score 0.39785403, normalized score: 0.5]
0.46 - Gunman opens fire on car just metres from scene of Hamid Sanambar murder [Hybrid (Result Set vector,hybridVector) Document fc8e153f-abb0-47c5-9a5d-1dfbe803f59f: original score 0.3897674, normalized score: 0.46209309]


[Description of how scoring works](https://weaviate.io/developers/weaviate/concepts/search/hybrid-search)

## A new method, RAG

RAG stands for Retrieval Augmented Generation. This is a recent trend in Information Retrieval that aims at reducing the problem of "hallucinations" for Large Language Model generation, and returns better answers based on local document archives.
- Traditional queries go as follows: the user makes a query to a search engine; the search engine returns, in some predefined format, the answer to that query.
- LLM queries: the user makes a query to a Large Language Model (LLM); the LLM creates an answer based on the (often unspecified) training data that was originally used to train it. The LLM often hallucinates, returing wrong answers.
- RAG: the user makes a query to a search engine; the search engine runs the query and gets its results. Before returning the results to the user, they are sent to a LLM to "process" and generate a textual response that is more convenient to read for the user, but (ideally) does not contain hallucinated information because they use precomputed (retrieved) results.

https://weaviate.io/developers/weaviate/model-providers

**GENERATIVE AI with OPENAI GPT-4**

Now let's try to include some generative AI prompts to this query (let's add context to the entities in the news, or let's translate them in Italian).
Note that this query will only work for those who have an openai paid module.

In [27]:
client.close()

cohere_key = userdata.get('COHERE_APIKEY')
openai_key = userdata.get("OPEN_APIKEY2")
headers = {
    "X-Cohere-Api-Key": cohere_key,
    "X-OpenAI-Api-Key": openai_key
}

client = weaviate.connect_to_embedded(headers=headers)

INFO:weaviate-client:Started /root/.cache/weaviate-embedded: process ID 1329


In [28]:
client.collections.delete_all()

client.collections.create(
    name="TestVectorizer",
    properties=[
        wc.Property(name="maintext", data_type=wc.DataType.TEXT, tokenization=Tokenization.WORD),
        wc.Property(name="title", data_type=wc.DataType.TEXT, tokenization=Tokenization.LOWERCASE),
        wc.Property(name="source", data_type=wc.DataType.TEXT, tokenization=Tokenization.FIELD),
        wc.Property(name="date", data_type=wc.DataType.DATE)
    ],
    vectorizer_config=[
        Configure.NamedVectors.text2vec_cohere(
            name="maintext_vector",
            source_properties=["maintext"],
            #model="embed-multilingual-light-v3.0"
        )
    ],
    generative_config=Configure.Generative.openai(model="gpt-4") # added generation module
)

In [29]:
documents = client.collections.get("TestVectorizer")
for doc in articles:
    documents.data.insert({
        "maintext": doc["maintext"],
        "title": doc["title"]
        }) # here weaviate performs the vectorization

In [30]:
response = documents.generate.near_text(
    query="sport",  # The model provider integration will automatically vectorize the query
    single_prompt="Write a short summary of maximum 100 characters in Italian of {maintext}",
    limit=2 # apply LLM to the top 2 results
)

In [31]:
for obj in response.objects:
    print(obj.properties["title"])
    print(f"Generated output: {obj.generated}")  # Note that the generated output is per object
    print("====================================================")
    print()

Leclerc dedicates win to Hubert
Generated output: Charles Leclerc ha ottenuto la sua prima vittoria in Formula Uno al Gran Premio del Belgio, dedicandola ad Anthoine Hubert.

Gunman opens fire on car just metres from scene of Hamid Sanambar murder
Generated output: La polizia cerca un uomo armato che ha sparato su un'auto a Dublino, vicino al luogo dove Hamid Sanambar è stato ucciso.



**GENERATIVE AI with an external COHERE**

The code above implements RAG using an external LLM module (OpenAI), invoked via the internal Weaviate module. We can also implement a RAG by calling the LLM directly, by using Cohere to implement the vectorization (inside Weaviate) and the generation (direcly with an API call). This way we do not need to pay for an OpenAI API key.

In [ ]:
!pip install cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 44.1 MB/s eta 0:00:00


In [ ]:
print("hybrid search: 'race'")
response = documents.query.hybrid(query="race", alpha=0.5, return_metadata=MetadataQuery(score=True, explain_score=True), limit=3)
for o in response.objects:
  print("{} - {} [{}]".format(round(o.metadata.score*100)/100, o.properties["title"],  o.metadata.explain_score.strip().replace("\n", '')))

hybrid search: 'race'
0.6 - Conte: 'Chelsea are not in the race to sign Sanchez' [Hybrid (Result Set keyword,bm25) Document 1fbd403d-8c83-4033-b991-9f302296a2f9: original score 1.1795324, normalized score: 0.5 - Hybrid (Result Set vector,hybridVector) Document 1fbd403d-8c83-4033-b991-9f302296a2f9: original score 0.31253493, normalized score: 0.10006716]
0.5 - Leclerc dedicates win to Hubert [Hybrid (Result Set keyword,bm25) Document 4ba90a36-2af6-4c47-b84e-565e5d9b6183: original score 0.46129686, normalized score: 0 - Hybrid (Result Set vector,hybridVector) Document 4ba90a36-2af6-4c47-b84e-565e5d9b6183: original score 0.3978603, normalized score: 0.5]
0.46 - Gunman opens fire on car just metres from scene of Hamid Sanambar murder [Hybrid (Result Set vector,hybridVector) Document 85a95143-ed92-4458-ba96-1c614389ae63: original score 0.389758, normalized score: 0.46202332]


In [ ]:
import cohere

co = cohere.ClientV2(api_key=cohere_key)
res = co.chat(
    model="command-r-plus-08-2024", # this is a cohere model
    messages=[
        {
            "role": "user",
            "content": "Write a short summary (100 characters at max), in Italian of the textual article \
            provided below: \n\n {}".format(response.objects[1].properties["maintext"]),
        } # response includes all the results returned by the previous hybrid query
    ],
)

print(res.message.content[0].text)

Charles Leclerc vince il Gran Premio del Belgio, dedicando la vittoria all'amico Anthoine Hubert, tragicamente scomparso.


In [ ]:
res = co.chat(
    model="command-r-plus-08-2024", # this is a cohere model
    messages=[
        {
            "role": "user",
            "content": "Write a one sentence summary in French of the textual article \
            provided below: \n\n {}".format(response.objects[0].properties["maintext"]),
        }
    ],
)

print(res.message.content[0].text)

Antonio Conte, entraîneur de Chelsea, a déclaré qu'il ne pensait pas que le club était en lice pour signer Alexis Sanchez, l'attaquant d'Arsenal, et a évité de discuter du marché des transferts, affirmant qu'il préférait laisser ces questions au club.


**BRING YOUR OWN VECTORS**

In the case you have your own vectors, then you can use Weaviare as a vectorDB

In [111]:
client.collections.delete_all()

client.collections.create(
    name="TestOwnVectors",

    properties=[
        wc.Property(name="maintext", data_type=wc.DataType.TEXT, tokenization=Tokenization.WORD),
        wc.Property(name="title", data_type=wc.DataType.TEXT, tokenization=Tokenization.LOWERCASE),
        wc.Property(name="source", data_type=wc.DataType.TEXT, tokenization=Tokenization.FIELD),
        wc.Property(name="date", data_type=wc.DataType.DATE)
    ],

    vectorizer_config=wc.Configure.Vectorizer.none(), # no vectorizer

    vector_index_config=Configure.VectorIndex.hnsw(

        # Distance metric
        #distance_metric=wc.VectorDistances.COSINE,
        distance_metric=wc.VectorDistances.L2_SQUARED,

        # Parameters for HNSW index construction
        #ef_construction=256,    # Dynamic list size during construction
        #max_connections=128,    # Maximum number of connections per node
        #quantizer=Configure.VectorIndex.Quantizer.bq(), # Quantizer configuration
        # Parameters for HNSW search
        #ef=-1,                  # Dynamic list size during search; -1 enables dynamic Ef
        #dynamic_ef_factor=15,   # Multiplier for dynamic Ef
        #dynamic_ef_min=200,     # Minimum threshold for dynamic Ef
        #dynamic_ef_max=1000,    # Maximum threshold for dynamic Ef
    )
)

documents = client.collections.get("TestOwnVectors")
for i, doc in enumerate(articles):
    documents.data.insert(
        {
        "maintext": doc["maintext"],
        "title": doc["title"]
        },
        vector=[i] * 2
        )

In [109]:
# If using named vectors, you can specify ones to include e.g. ['title', 'body'], or True to include all
for item in documents.iterator( include_vector=True ):
    print(json.dumps(item.properties, indent=2))
    print(item.vector)
    print("\n")

{
  "maintext": "The revival of supersonic passenger travel, thought to be long dead with the demise of Concorde nearly two decades ago, could be about to take wing as American Airlines has put in an order for 60 aircraft capable of flying at 1.7 times the speed of sound. \nBoom is a start-up based in Denver, Colorado, whose development of Overture, an ultra-fast successor to Concorde that seats 65 to 88 passengers, is so advanced that it showed off designs at last month's Farnborough air show.",
  "date": null,
  "title": "American Airlines orders 60 Overture supersonic jets",
  "source": null
}
{'default': [0.0, 0.0]}


{
  "maintext": "Charles Leclerc\nCharles Leclerc registered the maiden win of his Formula One career after romping to victory at the Belgian Grand Prix.\nLess than 24 hours after Leclerc's French motor racing contemporary, Anthoine Hubert, was killed at the Spa-Francorchamps venue, the young Monegasque driver delivered a dominant display to take the chequered flag in

In [110]:
#default distance is COSINE, we used the square of L2

response = documents.query.near_vector(
    near_vector=[1.0,1.0], # your query vector goes here
    return_metadata=MetadataQuery(distance=True,explain_score=True)
)

for o in response.objects:
    print("{} - {}".format(o.metadata.distance, o.properties["title"]))

0.0 - Conte: 'Chelsea are not in the race to sign Sanchez'
2.0 - Gunman opens fire on car just metres from scene of Hamid Sanambar murder
2.0 - American Airlines orders 60 Overture supersonic jets
8.0 - 'One-punch killer's sentence will make others think twice'
18.0 - Leclerc dedicates win to Hubert
